# Just Stand Up
Train a humanoid to stand up using reinforcement learning.

This notebook sets up the environment and tools, but **you'll need to implement the key logic**.

## 📦 Install dependencies

In [1]:
!git clone -q https://github.com/ajaykumaar/just_stand_up.git

In [2]:
!pip install -q dm_control
!pip install -q stable-baselines3
!pip install -q gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

## 🧠 Load the Humanoid environment (from dm_control)

In [1]:
from dm_control import suite

# Load humanoid standing task
env = suite.load(domain_name="humanoid", task_name="stand")

# Explore observation space
obs_spec = env.observation_spec()
print("Observation spec:", obs_spec)

# TODO: Wrap this env to make it compatible with stable-baselines3


/usr/local/lib/python3.11/dist-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


Observation spec: OrderedDict([('joint_angles', Array(shape=(21,), dtype=dtype('float64'), name='joint_angles')), ('head_height', Array(shape=(), dtype=dtype('float64'), name='head_height')), ('extremities', Array(shape=(12,), dtype=dtype('float64'), name='extremities')), ('torso_vertical', Array(shape=(3,), dtype=dtype('float64'), name='torso_vertical')), ('com_velocity', Array(shape=(3,), dtype=dtype('float64'), name='com_velocity')), ('velocity', Array(shape=(27,), dtype=dtype('float64'), name='velocity'))])


In [11]:
for obs_cat in list(obs_spec.keys()):
  print(obs_spec[obs_cat])

Array(shape=(21,), dtype=dtype('float64'), name='joint_angles')
Array(shape=(), dtype=dtype('float64'), name='head_height')
Array(shape=(12,), dtype=dtype('float64'), name='extremities')
Array(shape=(3,), dtype=dtype('float64'), name='torso_vertical')
Array(shape=(3,), dtype=dtype('float64'), name='com_velocity')
Array(shape=(27,), dtype=dtype('float64'), name='velocity')


In [12]:
21+1+12+3+3+27

67


 Observation space:
joint_angles: shape=(21,), sample=[ 0.36212236  0.38215001 -0.06353973 -0.26139594  0.49320975]...
head_height: shape=(), sample=[1.31775704]...
extremities: shape=(12,), sample=[-0.17623644  0.67599325 -0.10345743 -0.06086698 -0.04078925]...
torso_vertical: shape=(3,), sample=[-0.09338441  0.26695611 -0.95917349]...
com_velocity: shape=(3,), sample=[0. 0. 0.]...
velocity: shape=(27,), sample=[0. 0. 0. 0. 0.]...

 Action Space:
Shape: (21,), Minimum: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1.], Maximum: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### Visualize

In [2]:
!pip install -q opencv-python
# Step 1: System packages for EGL
!apt-get install -y libosmesa6-dev libgl1-mesa-glx libglfw3

# Step 2: Install mujoco + dm_control
!pip install mujoco==2.3.7
!pip install dm_control


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdrm-dev libgl-dev libglx-dev libosmesa6 libpciaccess-dev mesa-common-dev
Suggested packages:
  libgles1 libvulkan1
The following NEW packages will be installed:
  libdrm-dev libgl-dev libgl1-mesa-glx libglfw3 libglx-dev libosmesa6 libosmesa6-dev
  libpciaccess-dev mesa-common-dev
0 upgraded, 9 newly installed, 0 to remove and 30 not upgraded.
Need to get 5,850 kB of archives.
After this operation, 18.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpciaccess-dev amd64 0.16-3 [21.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libdrm-dev amd64 2.4.113-2~ubuntu0.22.04.1 [292 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgl-dev amd64 1.4.0-1 [101 kB]
Get:5

  Using cached mujoco-3.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (44 kB)
Using cached mujoco-3.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Attempting uninstall: mujoco
    Found existing installation: mujoco 2.3.7
    Uninstalling mujoco-2.3.7:
      Successfully uninstalled mujoco-2.3.7


In [1]:
import os
os.environ["MUJOCO_GL"] = "egl"

from dm_control import suite


In [2]:
env = suite.load(domain_name="humanoid", task_name="stand")
frame = env.physics.render(camera_id=0, height=240, width=320)


In [4]:
import cv2
import numpy as np
from dm_control import suite
from IPython.display import HTML
from base64 import b64encode

# Load the humanoid-stand environment
env = suite.load(domain_name="humanoid", task_name="stand")

# Video writer setup
video_path = "./humanoid_stand.avi"
width, height = 320, 240
fps = 30
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*"XVID"), fps, (width, height))

# Reset environment
ts = env.reset()

# Collect frames
for _ in range(150):  # 150 frames ~ 5 seconds
    action = np.random.uniform(low=env.action_spec().minimum,
                               high=env.action_spec().maximum,
                               size=env.action_spec().shape)
    ts = env.step(action)

    frame = env.physics.render(camera_id=0, height=height, width=width)
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    out.write(frame_bgr)

out.release()


## ⚙️ Define your Gym wrapper
_You'll need to convert `dm_control` into a Gym-compatible environment._

In [32]:
# !pip uninstall -y gym gymnasium
!pip install gym


Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Successfully uninstalled gym-0.25.2
Found existing installation: gymnasium 1.1.1
Uninstalling gymnasium-1.1.1:
  Successfully uninstalled gymnasium-1.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 35.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827726 sha256=b32f32e8c6b4ce8748e04b3af37a1be2de60ceeac04a1cc3f7003bf4d054dbdb
  Stored in directory: /root/.cache/pip/wheels/1c/77/9e/9af5470201a0b0543937933ee99ba884cd237d2faefe8f4d37
Successfully built gym
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, which is not installed.
dopamine-rl 4.1.2 requires gym<=0.25.2, b

In [2]:
for obs_cat in list(obs_spec.keys()):
  print(obs_spec[obs_cat])

Array(shape=(21,), dtype=dtype('float64'), name='joint_angles')
Array(shape=(), dtype=dtype('float64'), name='head_height')
Array(shape=(12,), dtype=dtype('float64'), name='extremities')
Array(shape=(3,), dtype=dtype('float64'), name='torso_vertical')
Array(shape=(3,), dtype=dtype('float64'), name='com_velocity')
Array(shape=(27,), dtype=dtype('float64'), name='velocity')


In [14]:
21+1+12+3+3+27

67

In [16]:
# Reset the environment and inspect the first observation
time_step = env.reset()
print("Observation space:")
for key, value in time_step.observation.items():
    print(f"{key}: shape={value.shape}, sample={value.ravel()[:5]}...")  # show a few values

# Print action spec
action_spec = env.action_spec()
print("\n Action Space:")
print(f"Shape: {action_spec.shape}, Minimum: {action_spec.minimum}, Maximum: {action_spec.maximum}")


Observation space:
joint_angles: shape=(21,), sample=[ 0.16468221 -0.48622888  0.09904348 -0.06312086 -0.31977614]...
head_height: shape=(), sample=[1.51069818]...
extremities: shape=(12,), sample=[0.23760685 0.1402834  0.20515201 0.16415527 0.27422706]...
torso_vertical: shape=(3,), sample=[-0.00881745  0.99837461  0.05630623]...
com_velocity: shape=(3,), sample=[0. 0. 0.]...
velocity: shape=(27,), sample=[0. 0. 0. 0. 0.]...

 Action Space:
Shape: (21,), Minimum: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1.], Maximum: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [23]:
gym.spaces.Box(low=-np.inf, high=np.inf, shape=(67,), dtype=np.float64 )

Box(-inf, inf, (67,), float64)

In [24]:
gym.spaces.Box(low=-1.0, high=1.0, shape=(21,), dtype=np.float64)

Box(-1.0, 1.0, (21,), float64)

In [5]:
!pip install -q gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 36.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


In [6]:
from stable_baselines3.common.env_checker import check_env

In [9]:
# TODO: Implement a wrapper that exposes reset(), step(), observation_space, action_space
# You may want to flatten the observation dict into a single np.ndarray

import gymnasium as gym
import numpy as np

class DMCWrapper(gym.Env):
    def __init__(self):
        self.env = suite.load(domain_name="humanoid", task_name="stand")
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(67,), dtype=np.float64 )
        self.action_space = gym.spaces.Box(low=-1.0, high=1.0, shape=(21,), dtype=np.float64)

    def reset(self, seed=None):
        ts = self.env.reset()
        return self._flatten_obs(ts.observation)

    def step(self, action):
        ts = self.env.step(action)
        obs = self._flatten_obs(ts.observation)
        reward = ts.reward or 0.0
        done = ts.last()
        return obs, reward, done, {}

    def _flatten_obs(self, obs):
        return np.concatenate([v.ravel() for v in obs.values()])
env = DMCWrapper()
check_env(env)

TypeError: The reset() method must accept a `seed` parameter

AssertionError: Your environment must inherit from the gymnasium.Env class cf. https://gymnasium.farama.org/api/env/

## 🤖 Train with PPO (optional starter agent in src/agents/ppo_agent.py)

In [ ]:
# TODO: Import your wrapped env and train with PPO
# from stable_baselines3 import PPO
# model = PPO('MlpPolicy', your_wrapped_env, verbose=1)
# model.learn(total_timesteps=...)

## ✅ Evaluate your trained policy

In [ ]:
# TODO: Load model, run inference loop, visualize behavior (if desired)
# obs = env.reset()
# for _ in range(1000):
#     action, _ = model.predict(obs)
#     obs, reward, done, info = env.step(action)
#     if done:
#         break

## 🧪 PPO Training on DummyEnv
Here's a working example of PPO training on a simple dummy environment.

In [11]:

import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

class DummyEnv(gym.Env):
    def __init__(self):
        super(DummyEnv, self).__init__()
        self.observation_space = gym.spaces.Box(low=-1, high=1, shape=(10,), dtype=np.float32)
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(4,), dtype=np.float32)

    def reset(self):
        return self.observation_space.sample()

    def step(self, action):
        obs = self.observation_space.sample()
        reward = np.random.rand()
        done = np.random.rand() > 0.95
        info = {}
        return obs, reward, done, info

env = DummyEnv()
check_env(env)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
print("✅ PPO training on DummyEnv completed!")


TypeError: The reset() method must accept a `seed` parameter